In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder
import random
from random import sample 
from pandas.io.json import json_normalize


## Training Set

In [2]:
# generate from original data
train_data = pd.read_csv('train.csv')
train_data.loc[:, 'timestamp'] = pd.to_datetime(train_data['timestamp'])
train_data.loc[:, 'event_code'] = train_data.event_code.astype('str')
train_data = train_data.sort_values(['installation_id', 'game_session', 'event_count'], ascending=True)
train_label = pd.read_csv('train_labels.csv').drop('title', axis=1)


In [3]:
random.seed(30)
sub_list = sample(list(train_label.installation_id.unique()), 10)

In [4]:
train_data = train_data.loc[train_data.installation_id.isin(sub_list)].reset_index(drop=True)
train_label = train_label.loc[train_label.installation_id.isin(sub_list)].reset_index(drop=True)


In [5]:
train_data.to_csv('train_sample.csv', index=False)
train_label.to_csv('train_label_sample.csv', index=False)

In [6]:
# read from sample data
train_data = pd.read_csv('train_sample.csv')
train_data.loc[:, 'timestamp'] = pd.to_datetime(train_data['timestamp'])
train_data.loc[:, 'event_code'] = train_data.event_code.astype('str')
train_data = train_data.sort_values(['installation_id', 'game_session', 'event_count'], ascending=True)
train_label = pd.read_csv('train_label_sample.csv')


In [7]:
train_data_copy = train_data.copy()
train_label_copy = train_label.copy()

### Columns: world_type 
In this session, by any given game session, we will count the total numbers of each type of game the kid has done. 

In [8]:
train_data = train_data_copy.copy()
train_label = train_label_copy.copy()

In [9]:
train_data_1 = train_data.drop('event_data', axis=1)

In [10]:
train_timestamp = train_data_1.groupby(['installation_id', 'game_session']).first().reset_index()
train_timestamp = train_timestamp.sort_values(['installation_id', 'timestamp'], ascending=True).reset_index(drop=True)

In [11]:
# train_label = train_label.merge(train_timestamp[['installation_id', 'game_session', 'timestamp']], 
#                                 how='left',
#                                 on=['installation_id', 'game_session'])
# train_label = train_label.sort_values(['installation_id', 'timestamp'], ascending=True).reset_index(drop=True)

In [12]:
# df_main = train_label.copy()

In [13]:
len(train_timestamp)

438

In [14]:
train_timestamp['world_type'] = train_timestamp['world']+'_'+train_timestamp['type']

In [15]:
world_type_dummy = pd.get_dummies(train_timestamp['world_type']).add_prefix('count_')
# train_timestamp
world_type_dummy.columns

Index(['count_CRYSTALCAVES_Activity', 'count_CRYSTALCAVES_Assessment',
       'count_CRYSTALCAVES_Clip', 'count_CRYSTALCAVES_Game',
       'count_MAGMAPEAK_Activity', 'count_MAGMAPEAK_Assessment',
       'count_MAGMAPEAK_Clip', 'count_MAGMAPEAK_Game', 'count_NONE_Clip',
       'count_TREETOPCITY_Activity', 'count_TREETOPCITY_Assessment',
       'count_TREETOPCITY_Clip', 'count_TREETOPCITY_Game'],
      dtype='object')

In [16]:
train_timestamp[world_type_dummy.columns] = world_type_dummy

In [17]:
train_timestamp[world_type_dummy.columns] = (train_timestamp
                   .groupby(['installation_id'])[world_type_dummy.columns]
                   .transform(lambda x: x.cumsum() - x))

In [18]:
train_timestamp.shape

(438, 24)

### Columns: game_time by world and type 
In this session, by any given game session, we will count the total game time of each type of game the kid has done. 

In [19]:
train_data_2 = (train_data
                .groupby(['installation_id', 'game_session'])
                .last()
                .drop('event_data', axis=1)
                .reset_index())


In [20]:
train_data_2['world_type'] = train_data_2['world']+'_'+train_data_2['type']

In [21]:
train_time_cum = pd.get_dummies(train_data_2.world_type).multiply(train_data_2.game_time, axis="index").add_prefix('time_')

In [22]:
train_data_2[train_time_cum.columns] = train_time_cum

In [23]:
train_data_2 = (train_data_2
                .drop('timestamp', axis=1)
                .merge(train_timestamp[['installation_id', 'game_session', 'timestamp']], 
                        on=['installation_id', 'game_session'], 
                        how='right')
                .sort_values(['installation_id', 'timestamp'], ascending=True)
                .reset_index(drop=True))
train_data_2.shape

(438, 24)

In [24]:
train_data_2[train_time_cum.columns] = (train_data_2
                                        .groupby(['installation_id'])[train_time_cum.columns]
                                        .transform(lambda x: x.cumsum() - x))


In [25]:
train_data_2.shape

(438, 24)

### Accuracy: deal with the event_data


In [26]:
assessments_four = ['Cart Balancer (Assessment)', 
                    'Cauldron Filler (Assessment)', 
                    'Chest Sorter (Assessment)', 
                    'Mushroom Sorter (Assessment)']

assessments_one = ['Bird Measurer (Assessment)']


In [27]:
import ast
from pandas.io.json import json_normalize

def only_dict(d):
    '''
    Convert json string representation of dictionary to a python dict
    '''
    try:
        return json.loads(d)['correct']
    except:
        return np.nan


In [28]:
train_data_3 = train_data.copy()
train_data_3['assessment'] = train_data['event_data'].apply(only_dict)

In [29]:
train_data_3['assessment'] = (train_data_3
                                   .where(((train_data_3.event_code == '4100') & (train_data_3.title.isin(assessments_four))) |
                                         ((train_data_3.event_code == '4110') & (train_data_3.title.isin(assessments_one))), np.nan))['assessment']

In [30]:
train_data_3 = train_data_3.drop('event_data', axis=1)

In [31]:
train_assessment = pd.get_dummies(train_data_3.assessment).add_prefix('assessment_')

In [32]:
train_data_3[train_assessment.columns] = train_assessment

In [33]:
train_data_4 = train_data_3.groupby(['installation_id', 'game_session']).sum().reset_index()


In [34]:
train_data_4 = train_data_4.merge(train_timestamp[['installation_id', 'game_session', 
                                                   'timestamp']], 
                                  on=['installation_id', 'game_session'])

In [35]:
train_data_4[train_assessment.columns] = (train_data_4
                                          .sort_values(['installation_id', 'timestamp'], ascending=True)
                                          .groupby(['installation_id'])[train_assessment.columns]
                                          .transform(lambda x: x.cumsum() - x))
train_data_4.shape

(438, 7)

In [36]:
train_data_same_assess = train_data_3.groupby(['installation_id', 'game_session']).sum().reset_index()
train_data_same_assess = train_data_same_assess.merge(train_timestamp[['installation_id', 'game_session', 
                                                   'timestamp', 'title']], 
                                  on=['installation_id', 'game_session'])

In [37]:
train_data_same_assess[[col+'_same_assess' for col in train_assessment.columns]] = (train_data_same_assess
                                          .sort_values(['installation_id', 'timestamp'], ascending=True)
                                          .groupby(['installation_id', 'title'])[train_assessment.columns]
                                          .transform(lambda x: x.cumsum() - x))
train_data_same_assess

,installation_id,game_session,event_count,game_time,assessment_False,assessment_True,timestamp,title,assessment_False_same_assess,assessment_True_same_assess
0,07c25408,1c5ff4f9d45bddd1,12561,9528176,5,0,2019-08-31 16:40:38.102000+00:00,Chest Sorter (Assessment),0,0
1,07c25408,361e9798f46c800d,66,80835,0,0,2019-08-31 16:38:30.459000+00:00,Pan Balance,0,0
2,07c25408,51e0076ea0f92b45,1,0,0,0,2019-08-31 16:31:40.005000+00:00,Crystal Caves - Level 3,0,0
3,07c25408,a715ac9895766515,91,179508,0,0,2019-08-31 16:37:28.782000+00:00,Egg Dropper (Activity),0,0
4,07c25408,a8da31bf8c4f5793,1,0,0,0,2019-08-31 16:30:05.119000+00:00,Crystal Caves - Level 3,0,0
5,07c25408,a8da571f08bc70b1,7021,14981881,0,0,2019-08-31 16:32:30.819000+00:00,Egg Dropper (Activity),0,0
6,3bd55510,08f3c0e85eedde1b,1,0,0,0,2019-07-25 18:13:09.690000+00:00,Ordering Spheres,0,0
7,3bd55510,0c39f69acf07b328,2415,2314015,0,0,2019-07-25 22:11:29.726000+00:00,Crystals Rule,0,0
8,3bd55510,184e03508aa9d8f7,1,0,0,0,2019-07-25 18:12:33.233000+00:00,Welcome to Lost Lagoon!,0,0
9,3bd55510,1d9868fa570106a3,1,0,0,0,2019-09-06 16:29:27.838000+00:00,Ordering Spheres,0,0


In [38]:
train_data_4 = (train_data_4[['installation_id', 'game_session', 'assessment_False', 'assessment_True']]
                .merge(train_data_same_assess[['installation_id', 'game_session', 'assessment_False_same_assess', 'assessment_True_same_assess']], 
                       on=['installation_id', 'game_session']))
train_data_4

,installation_id,game_session,assessment_False,assessment_True,assessment_False_same_assess,assessment_True_same_assess
0,07c25408,1c5ff4f9d45bddd1,0,0,0,0
1,07c25408,361e9798f46c800d,0,0,0,0
2,07c25408,51e0076ea0f92b45,0,0,0,0
3,07c25408,a715ac9895766515,0,0,0,0
4,07c25408,a8da31bf8c4f5793,0,0,0,0
5,07c25408,a8da571f08bc70b1,0,0,0,0
6,3bd55510,08f3c0e85eedde1b,0,0,0,0
7,3bd55510,0c39f69acf07b328,0,1,0,0
8,3bd55510,184e03508aa9d8f7,0,0,0,0
9,3bd55510,1d9868fa570106a3,0,2,0,0


In [39]:
train_data_4.shape

(438, 6)

### Deal with 5 hour staff

In [40]:
# count how many same assessments true and false in history, 
train_data_5_hour = train_data_3.groupby(['installation_id', 'game_session']).sum().reset_index()
train_data_5_hour = train_data_5_hour.merge(train_timestamp[['installation_id', 'game_session', 'timestamp', 'title']], 
                                            on=['installation_id', 'game_session'])
train_data_5_hour = train_data_5_hour.sort_values(['installation_id', 'timestamp'])[['installation_id', 'game_session', 
       'assessment_False', 'assessment_True', 'timestamp', 'title']]

train_data_5_hour_merged = train_data_5_hour.merge(train_data_5_hour, on='installation_id')
train_data_5_hour_merged = train_data_5_hour_merged.loc[(train_data_5_hour_merged.timestamp_x > train_data_5_hour_merged.timestamp_y)&
                             (train_data_5_hour_merged.timestamp_x <= train_data_5_hour_merged.timestamp_y + pd.offsets.Hour(5))&
                             (train_data_5_hour_merged.title_x == train_data_5_hour_merged.title_y)]
train_data_5_hour_merged = train_data_5_hour_merged.groupby(['installation_id', 'game_session_x']).sum()


train_data_5_hour_merged = train_data_5_hour_merged.reset_index()[['installation_id', 'game_session_x', 
                                                                   'assessment_False_y', 'assessment_True_y']]


train_data_5_hour_merged.columns = ['installation_id', 'game_session', 
                                    'assessment_False_five_hours', 'assessment_True_five_hours']

train_data_5_hour_merged = train_data_5_hour_merged.merge(train_timestamp, 
                                                          on=['installation_id', 'game_session'], 
                                                          how='right')

train_data_5_hour_merged = train_data_5_hour_merged.fillna(0)
train_data_5_hour_merged = train_data_5_hour_merged[['installation_id', 'game_session', 'assessment_False_five_hours',
       'assessment_True_five_hours', 'timestamp']].sort_values(['installation_id', 'timestamp']).reset_index(drop=True)

In [41]:
train_data_5_hour_merged

,installation_id,game_session,assessment_False_five_hours,assessment_True_five_hours,timestamp
0,07c25408,a8da31bf8c4f5793,0.0,0.0,2019-08-31 16:30:05.119000+00:00
1,07c25408,51e0076ea0f92b45,0.0,0.0,2019-08-31 16:31:40.005000+00:00
2,07c25408,a8da571f08bc70b1,0.0,0.0,2019-08-31 16:32:30.819000+00:00
3,07c25408,a715ac9895766515,0.0,0.0,2019-08-31 16:37:28.782000+00:00
4,07c25408,361e9798f46c800d,0.0,0.0,2019-08-31 16:38:30.459000+00:00
5,07c25408,1c5ff4f9d45bddd1,0.0,0.0,2019-08-31 16:40:38.102000+00:00
6,3bd55510,184e03508aa9d8f7,0.0,0.0,2019-07-25 18:12:33.233000+00:00
7,3bd55510,87d6dac6c6ea513d,0.0,0.0,2019-07-25 18:12:44.677000+00:00
8,3bd55510,08f3c0e85eedde1b,0.0,0.0,2019-07-25 18:13:09.690000+00:00
9,3bd55510,623b915ea4ff72cc,0.0,0.0,2019-07-25 18:13:23.601000+00:00


In [42]:
# count how many same assessments in history, 
train_data_5_hour = train_data_3.groupby(['installation_id', 'game_session']).sum().reset_index()
train_data_5_hour = train_data_5_hour.merge(train_timestamp[['installation_id', 'game_session', 'timestamp', 'title']], 
                                            on=['installation_id', 'game_session'])
train_data_5_hour = train_data_5_hour.sort_values(['installation_id', 'timestamp'])[['installation_id', 'game_session', 
       'assessment_False', 'assessment_True', 'timestamp', 'title']]

train_data_5_hour_count = train_data_5_hour.merge(train_data_5_hour, on='installation_id')
train_data_5_hour_count = train_data_5_hour_count.loc[(train_data_5_hour_count.timestamp_x > train_data_5_hour_count.timestamp_y)&
                             (train_data_5_hour_count.timestamp_x <= train_data_5_hour_count.timestamp_y + pd.offsets.Hour(5))&
                             (train_data_5_hour_count.title_x == train_data_5_hour_count.title_y)]
train_data_5_hour_count = train_data_5_hour_count.groupby(['installation_id', 'game_session_x']).count()


train_data_5_hour_count = train_data_5_hour_count.reset_index()[['installation_id', 'game_session_x', 
                                                                   'assessment_False_y']]


train_data_5_hour_count.columns = ['installation_id', 'game_session', 
                                    'assessment_five_hours_count']

train_data_5_hour_count = train_data_5_hour_count.merge(train_timestamp, 
                                                          on=['installation_id', 'game_session'], 
                                                          how='right')

train_data_5_hour_count = train_data_5_hour_count.fillna(0)
train_data_5_hour_count = train_data_5_hour_count[['installation_id', 'game_session', 'assessment_five_hours_count', 'timestamp']].sort_values(['installation_id', 'timestamp']).reset_index(drop=True)

In [43]:
train_data_5_hour_count

,installation_id,game_session,assessment_five_hours_count,timestamp
0,07c25408,a8da31bf8c4f5793,0.0,2019-08-31 16:30:05.119000+00:00
1,07c25408,51e0076ea0f92b45,1.0,2019-08-31 16:31:40.005000+00:00
2,07c25408,a8da571f08bc70b1,0.0,2019-08-31 16:32:30.819000+00:00
3,07c25408,a715ac9895766515,1.0,2019-08-31 16:37:28.782000+00:00
4,07c25408,361e9798f46c800d,0.0,2019-08-31 16:38:30.459000+00:00
5,07c25408,1c5ff4f9d45bddd1,0.0,2019-08-31 16:40:38.102000+00:00
6,3bd55510,184e03508aa9d8f7,0.0,2019-07-25 18:12:33.233000+00:00
7,3bd55510,87d6dac6c6ea513d,0.0,2019-07-25 18:12:44.677000+00:00
8,3bd55510,08f3c0e85eedde1b,0.0,2019-07-25 18:13:09.690000+00:00
9,3bd55510,623b915ea4ff72cc,0.0,2019-07-25 18:13:23.601000+00:00


### Column: event code and counts

In [44]:
train_data_5 = train_data.copy().drop('event_data', axis=1)

In [45]:
train_event_code = pd.get_dummies(train_data_5.event_code).add_prefix('event_code_')
train_data_5[train_event_code.columns] = train_event_code

In [46]:
train_data_5_event = train_data_5.groupby(['installation_id', 'game_session']).sum().reset_index()
train_data_5 = train_timestamp.merge(train_data_5_event, on=['installation_id', 'game_session']) 
train_data_5 = train_data_5.sort_values(['installation_id', 'timestamp'], ascending=True)



In [47]:
train_data_5[train_event_code.columns] = (train_data_5
                                          .sort_values(['installation_id', 'timestamp'], ascending=True)
                                          .groupby(['installation_id'])[train_event_code.columns]
                                          .transform(lambda x: x.cumsum() - x))


In [48]:
train_data_5.shape

(438, 67)

### Column: by title

In [49]:
train_title = pd.get_dummies(train_timestamp['title']).add_prefix('title_')
train_title.columns

Index(['title_12 Monkeys', 'title_Air Show', 'title_All Star Sorting',
       'title_Balancing Act', 'title_Bird Measurer (Assessment)',
       'title_Bottle Filler (Activity)', 'title_Bubble Bath',
       'title_Bug Measurer (Activity)', 'title_Cart Balancer (Assessment)',
       'title_Cauldron Filler (Assessment)', 'title_Chest Sorter (Assessment)',
       'title_Chicken Balancer (Activity)', 'title_Chow Time',
       'title_Costume Box', 'title_Crystal Caves - Level 1',
       'title_Crystal Caves - Level 2', 'title_Crystal Caves - Level 3',
       'title_Crystals Rule', 'title_Dino Dive', 'title_Dino Drink',
       'title_Egg Dropper (Activity)', 'title_Fireworks (Activity)',
       'title_Flower Waterer (Activity)', 'title_Happy Camel',
       'title_Heavy, Heavier, Heaviest', 'title_Honey Cake',
       'title_Leaf Leader', 'title_Lifting Heavy Things',
       'title_Magma Peak - Level 1', 'title_Magma Peak - Level 2',
       'title_Mushroom Sorter (Assessment)', 'title_Ordering 

In [50]:
train_data_title = train_timestamp.copy()

In [51]:
train_data_title[train_title.columns] = train_title

In [52]:
train_data_title[train_title.columns] = (train_data_title
                   .groupby(['installation_id'])[train_title.columns]
                   .transform(lambda x: x.cumsum() - x))

In [53]:
train_data_title.columns

Index(['installation_id', 'game_session', 'event_id', 'timestamp',
       'event_count', 'event_code', 'game_time', 'title', 'type', 'world',
       'world_type', 'count_CRYSTALCAVES_Activity',
       'count_CRYSTALCAVES_Assessment', 'count_CRYSTALCAVES_Clip',
       'count_CRYSTALCAVES_Game', 'count_MAGMAPEAK_Activity',
       'count_MAGMAPEAK_Assessment', 'count_MAGMAPEAK_Clip',
       'count_MAGMAPEAK_Game', 'count_NONE_Clip', 'count_TREETOPCITY_Activity',
       'count_TREETOPCITY_Assessment', 'count_TREETOPCITY_Clip',
       'count_TREETOPCITY_Game', 'title_12 Monkeys', 'title_Air Show',
       'title_All Star Sorting', 'title_Balancing Act',
       'title_Bird Measurer (Assessment)', 'title_Bottle Filler (Activity)',
       'title_Bubble Bath', 'title_Bug Measurer (Activity)',
       'title_Cart Balancer (Assessment)',
       'title_Cauldron Filler (Assessment)', 'title_Chest Sorter (Assessment)',
       'title_Chicken Balancer (Activity)', 'title_Chow Time',
       'title_Costum

In [54]:
train_title.shape

(438, 44)

### Column: coverage of games

In [55]:
title_over = []
set_NONE = set(['Welcome to Lost Lagoon!'])
set_MAGMAPEAK = set(['Magma Peak - Level 1', 'Sandcastle Builder (Activity)',
                                       'Slop Problem', 'Scrub-A-Dub', 'Watering Hole (Activity)',
                                       'Magma Peak - Level 2', 'Dino Drink', 'Bubble Bath',
                                     'Bottle Filler (Activity)', 'Dino Dive',
                                       'Cauldron Filler (Assessment)'])
set_TREETOPCITY = set(['Tree Top City - Level 1', 'Ordering Spheres', 'All Star Sorting',
                                       'Costume Box', 'Fireworks (Activity)', '12 Monkeys',
                                       'Tree Top City - Level 2', 'Flower Waterer (Activity)',
                                       "Pirate's Tale", 'Mushroom Sorter (Assessment)', 'Air Show',
                                       'Treasure Map', 'Tree Top City - Level 3', 'Crystals Rule',
                                       'Rulers', 'Bug Measurer (Activity)', 'Bird Measurer (Assessment)'])
set_CRYSTALCAVES = set(['Crystal Caves - Level 1', 'Crystal Caves - Level 2',
                                       'Crystal Caves - Level 3', 'Cart Balancer (Assessment)',
                                       'Egg Dropper (Activity)', 'Chow Time', 'Leaf Leader',
                                       'Pan Balance', 'Chest Sorter (Assessment)', 'Balancing Act',
                                       'Chicken Balancer (Activity)', 'Lifting Heavy Things',
                                       'Honey Cake', 'Happy Camel', 'Heavy, Heavier, Heaviest'])
for row in train_timestamp.world:
    if row == 'NONE':
        title_over.append(set_NONE)
    elif row == 'MAGMAPEAK':
        title_over.append(set_MAGMAPEAK)
    elif row == 'TREETOPCITY':
        title_over.append(set_TREETOPCITY)
    elif row == 'CRYSTALCAVES':
        title_over.append(set_CRYSTALCAVES)

train_coverage = train_timestamp.copy()
train_coverage['title_over'] = title_over

train_coverage

,installation_id,game_session,event_id,timestamp,event_count,event_code,game_time,title,type,world,...,count_MAGMAPEAK_Activity,count_MAGMAPEAK_Assessment,count_MAGMAPEAK_Clip,count_MAGMAPEAK_Game,count_NONE_Clip,count_TREETOPCITY_Activity,count_TREETOPCITY_Assessment,count_TREETOPCITY_Clip,count_TREETOPCITY_Game,title_over
0,07c25408,a8da31bf8c4f5793,27253bdc,2019-08-31 16:30:05.119000+00:00,1,2000,0,Crystal Caves - Level 3,Clip,CRYSTALCAVES,...,0,0,0,0,0,0,0,0,0,"{Pan Balance, Crystal Caves - Level 3, Lifting..."
1,07c25408,51e0076ea0f92b45,27253bdc,2019-08-31 16:31:40.005000+00:00,1,2000,0,Crystal Caves - Level 3,Clip,CRYSTALCAVES,...,0,0,0,0,0,0,0,0,0,"{Pan Balance, Crystal Caves - Level 3, Lifting..."
2,07c25408,a8da571f08bc70b1,9b23e8ee,2019-08-31 16:32:30.819000+00:00,1,2000,0,Egg Dropper (Activity),Activity,CRYSTALCAVES,...,0,0,0,0,0,0,0,0,0,"{Pan Balance, Crystal Caves - Level 3, Lifting..."
3,07c25408,a715ac9895766515,9b23e8ee,2019-08-31 16:37:28.782000+00:00,1,2000,0,Egg Dropper (Activity),Activity,CRYSTALCAVES,...,0,0,0,0,0,0,0,0,0,"{Pan Balance, Crystal Caves - Level 3, Lifting..."
4,07c25408,361e9798f46c800d,9c5ef70c,2019-08-31 16:38:30.459000+00:00,1,2000,0,Pan Balance,Game,CRYSTALCAVES,...,0,0,0,0,0,0,0,0,0,"{Pan Balance, Crystal Caves - Level 3, Lifting..."
5,07c25408,1c5ff4f9d45bddd1,5b49460a,2019-08-31 16:40:38.102000+00:00,1,2000,0,Chest Sorter (Assessment),Assessment,CRYSTALCAVES,...,0,0,0,0,0,0,0,0,0,"{Pan Balance, Crystal Caves - Level 3, Lifting..."
6,3bd55510,184e03508aa9d8f7,27253bdc,2019-07-25 18:12:33.233000+00:00,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE,...,0,0,0,0,0,0,0,0,0,{Welcome to Lost Lagoon!}
7,3bd55510,87d6dac6c6ea513d,27253bdc,2019-07-25 18:12:44.677000+00:00,1,2000,0,Tree Top City - Level 1,Clip,TREETOPCITY,...,0,0,0,0,1,0,0,0,0,"{Treasure Map, All Star Sorting, Bug Measurer ..."
8,3bd55510,08f3c0e85eedde1b,27253bdc,2019-07-25 18:13:09.690000+00:00,1,2000,0,Ordering Spheres,Clip,TREETOPCITY,...,0,0,0,0,1,0,0,1,0,"{Treasure Map, All Star Sorting, Bug Measurer ..."
9,3bd55510,623b915ea4ff72cc,b7dc8128,2019-07-25 18:13:23.601000+00:00,1,2000,0,All Star Sorting,Game,TREETOPCITY,...,0,0,0,0,1,0,0,2,0,"{Treasure Map, All Star Sorting, Bug Measurer ..."


In [56]:
train_coverage.columns

Index(['installation_id', 'game_session', 'event_id', 'timestamp',
       'event_count', 'event_code', 'game_time', 'title', 'type', 'world',
       'world_type', 'count_CRYSTALCAVES_Activity',
       'count_CRYSTALCAVES_Assessment', 'count_CRYSTALCAVES_Clip',
       'count_CRYSTALCAVES_Game', 'count_MAGMAPEAK_Activity',
       'count_MAGMAPEAK_Assessment', 'count_MAGMAPEAK_Clip',
       'count_MAGMAPEAK_Game', 'count_NONE_Clip', 'count_TREETOPCITY_Activity',
       'count_TREETOPCITY_Assessment', 'count_TREETOPCITY_Clip',
       'count_TREETOPCITY_Game', 'title_over'],
      dtype='object')

In [57]:
# count how many same assessments true and false in history, 
train_coverage = train_coverage[['installation_id', 'game_session', 'timestamp', 'title', 'title_over']]

train_coverage_merged = train_coverage.merge(train_coverage, on='installation_id')
train_coverage_merged = train_coverage_merged.loc[train_coverage_merged.timestamp_x > train_coverage_merged.timestamp_y]
train_coverage_merged = train_coverage_merged.groupby(['installation_id', 'game_session_x']).title_y.unique().reset_index()


train_coverage_merged = train_coverage_merged[['installation_id', 'game_session_x', 
                                                                   'title_y']]


train_coverage_merged.columns = ['installation_id', 'game_session', 'title_set']
train_coverage_merged['title_set'] = [set(row) for row in train_coverage_merged['title_set']]

train_coverage_merged = train_coverage_merged.merge(train_coverage, 
                                                          on=['installation_id', 'game_session'], 
                                                          how='right')
train_coverage_merged = train_coverage_merged.fillna(np.nan)

train_coverage_merged = train_coverage_merged.sort_values(['installation_id', 'timestamp']).reset_index(drop=True)

In [58]:
index_empty_title_set = train_coverage_merged.loc[train_coverage_merged.title_set.isnull(), :].index
index_non_empty_title_set = train_coverage_merged.loc[~train_coverage_merged.title_set.isnull(), :].index

In [59]:
train_coverage_merged.loc[index_empty_title_set, 'non_coverage'] = train_coverage_merged.loc[index_empty_title_set, 'title_over']
train_coverage_merged.loc[index_non_empty_title_set, 'non_coverage'] = train_coverage_merged.loc[index_non_empty_title_set, 'title_over'] - train_coverage_merged.loc[index_non_empty_title_set, 'title_set']
train_coverage_merged

,installation_id,game_session,title_set,timestamp,title,title_over,non_coverage
0,07c25408,a8da31bf8c4f5793,NaN,2019-08-31 16:30:05.119000+00:00,Crystal Caves - Level 3,"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Crystal Caves - Level 3, Lifting..."
1,07c25408,51e0076ea0f92b45,{Crystal Caves - Level 3},2019-08-31 16:31:40.005000+00:00,Crystal Caves - Level 3,"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came..."
2,07c25408,a8da571f08bc70b1,{Crystal Caves - Level 3},2019-08-31 16:32:30.819000+00:00,Egg Dropper (Activity),"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came..."
3,07c25408,a715ac9895766515,"{Crystal Caves - Level 3, Egg Dropper (Activity)}",2019-08-31 16:37:28.782000+00:00,Egg Dropper (Activity),"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came..."
4,07c25408,361e9798f46c800d,"{Crystal Caves - Level 3, Egg Dropper (Activity)}",2019-08-31 16:38:30.459000+00:00,Pan Balance,"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came..."
5,07c25408,1c5ff4f9d45bddd1,"{Crystal Caves - Level 3, Egg Dropper (Activit...",2019-08-31 16:40:38.102000+00:00,Chest Sorter (Assessment),"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Happy Camel, Chest Sorter (Assessment), Balan..."
6,3bd55510,184e03508aa9d8f7,NaN,2019-07-25 18:12:33.233000+00:00,Welcome to Lost Lagoon!,{Welcome to Lost Lagoon!},{Welcome to Lost Lagoon!}
7,3bd55510,87d6dac6c6ea513d,{Welcome to Lost Lagoon!},2019-07-25 18:12:44.677000+00:00,Tree Top City - Level 1,"{Treasure Map, All Star Sorting, Bug Measurer ...","{Tree Top City - Level 2, Crystals Rule, Tree ..."
8,3bd55510,08f3c0e85eedde1b,"{Tree Top City - Level 1, Welcome to Lost Lago...",2019-07-25 18:13:09.690000+00:00,Ordering Spheres,"{Treasure Map, All Star Sorting, Bug Measurer ...","{Tree Top City - Level 2, Crystals Rule, Firew..."
9,3bd55510,623b915ea4ff72cc,"{Tree Top City - Level 1, Ordering Spheres, We...",2019-07-25 18:13:23.601000+00:00,All Star Sorting,"{Treasure Map, All Star Sorting, Bug Measurer ...","{Tree Top City - Level 2, Crystals Rule, Firew..."


In [60]:
train_coverage_merged['coverage_ratio'] = [1-len(train_coverage_merged.non_coverage[i])/len(train_coverage_merged.title_over[i]) 
                                           for i in range(len(train_coverage_merged))]
train_coverage_merged

,installation_id,game_session,title_set,timestamp,title,title_over,non_coverage,coverage_ratio
0,07c25408,a8da31bf8c4f5793,NaN,2019-08-31 16:30:05.119000+00:00,Crystal Caves - Level 3,"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Crystal Caves - Level 3, Lifting...",0.000000
1,07c25408,51e0076ea0f92b45,{Crystal Caves - Level 3},2019-08-31 16:31:40.005000+00:00,Crystal Caves - Level 3,"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came...",0.066667
2,07c25408,a8da571f08bc70b1,{Crystal Caves - Level 3},2019-08-31 16:32:30.819000+00:00,Egg Dropper (Activity),"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came...",0.066667
3,07c25408,a715ac9895766515,"{Crystal Caves - Level 3, Egg Dropper (Activity)}",2019-08-31 16:37:28.782000+00:00,Egg Dropper (Activity),"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came...",0.133333
4,07c25408,361e9798f46c800d,"{Crystal Caves - Level 3, Egg Dropper (Activity)}",2019-08-31 16:38:30.459000+00:00,Pan Balance,"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Pan Balance, Lifting Heavy Things, Happy Came...",0.133333
5,07c25408,1c5ff4f9d45bddd1,"{Crystal Caves - Level 3, Egg Dropper (Activit...",2019-08-31 16:40:38.102000+00:00,Chest Sorter (Assessment),"{Pan Balance, Crystal Caves - Level 3, Lifting...","{Happy Camel, Chest Sorter (Assessment), Balan...",0.200000
6,3bd55510,184e03508aa9d8f7,NaN,2019-07-25 18:12:33.233000+00:00,Welcome to Lost Lagoon!,{Welcome to Lost Lagoon!},{Welcome to Lost Lagoon!},0.000000
7,3bd55510,87d6dac6c6ea513d,{Welcome to Lost Lagoon!},2019-07-25 18:12:44.677000+00:00,Tree Top City - Level 1,"{Treasure Map, All Star Sorting, Bug Measurer ...","{Tree Top City - Level 2, Crystals Rule, Tree ...",0.000000
8,3bd55510,08f3c0e85eedde1b,"{Tree Top City - Level 1, Welcome to Lost Lago...",2019-07-25 18:13:09.690000+00:00,Ordering Spheres,"{Treasure Map, All Star Sorting, Bug Measurer ...","{Tree Top City - Level 2, Crystals Rule, Firew...",0.058824
9,3bd55510,623b915ea4ff72cc,"{Tree Top City - Level 1, Ordering Spheres, We...",2019-07-25 18:13:23.601000+00:00,All Star Sorting,"{Treasure Map, All Star Sorting, Bug Measurer ...","{Tree Top City - Level 2, Crystals Rule, Firew...",0.117647


In [61]:
train_coverage_merged = train_coverage_merged[['installation_id', 'game_session', 'coverage_ratio']]
train_coverage_merged

,installation_id,game_session,coverage_ratio
0,07c25408,a8da31bf8c4f5793,0.000000
1,07c25408,51e0076ea0f92b45,0.066667
2,07c25408,a8da571f08bc70b1,0.066667
3,07c25408,a715ac9895766515,0.133333
4,07c25408,361e9798f46c800d,0.133333
5,07c25408,1c5ff4f9d45bddd1,0.200000
6,3bd55510,184e03508aa9d8f7,0.000000
7,3bd55510,87d6dac6c6ea513d,0.000000
8,3bd55510,08f3c0e85eedde1b,0.058824
9,3bd55510,623b915ea4ff72cc,0.117647


### Column: deal with timestamp

In [62]:
time_period = []
for time in train_timestamp.timestamp:
    if (time.hour < 6) | (time.hour >= 22):
        time_period.append('midnight')
    elif (time.hour >= 6) & (time.hour < 12):
        time_period.append('morning')
    elif (time.hour >=12) & (time.hour < 18):
        time_period.append('afternoon')
    elif (time.hour >=18) & (time.hour < 22):
        time_period.append('night')
time_period

['afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'afternoon',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'night',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midnight',
 'midn

In [63]:
train_time_period = train_timestamp.copy()
train_time_period['time_period'] = time_period

In [64]:
train_time_period = pd.concat([train_time_period, pd.get_dummies(train_time_period['time_period']).add_prefix('time_period_')], axis=1)

In [66]:
train_time_period = train_time_period[['installation_id', 'game_session', 
                                       'time_period_afternoon', 'time_period_midnight', 
#                                        'time_period_morning',
                                       'time_period_night']]

In [67]:
train_time_period.shape

(438, 5)

### Combine them together

In [80]:
# time columns
train_final = train_timestamp.merge(train_data_2[['installation_id', 'game_session']+list(train_time_cum.columns)], 
                                    on=['installation_id', 'game_session'])
# how many assessments in history and how many same assessments in history
train_final = train_final.merge(train_data_4, 
                                on=['installation_id', 'game_session'])

# event code
train_final = train_final.merge(train_data_5[['installation_id', 'game_session']+list(train_event_code.columns)], 
                                on=['installation_id', 'game_session'])

# the same assessments: how many true and false and the counts
train_final = train_final.merge(train_data_5_hour_merged[['installation_id', 'game_session', 'assessment_False_five_hours',
       'assessment_True_five_hours']], 
                                on=['installation_id', 'game_session'])

train_final = train_final.merge(train_data_5_hour_count[['installation_id', 'game_session', 'assessment_five_hours_count']], 
                                on=['installation_id', 'game_session'])

# column: add titles
train_final = train_final.merge(train_data_title[['installation_id', 'game_session']+list(train_title.columns)], 
                                on=['installation_id', 'game_session'])

# add game coverage ratio
train_final = train_final.merge(train_coverage_merged, 
                                on=['installation_id', 'game_session'])


# process the timestamp colum
train_final = train_final.merge(train_time_period, 
                                on=['installation_id', 'game_session'])


# merge with the label
train_final = train_final.merge(train_label,
                                on=['installation_id', 'game_session'], how='right')

# calculate the average time spent in each game type of each world
for i in range(len(train_time_cum.columns)):
    train_final[train_time_cum.columns[i]+'_avg'] = train_final[train_time_cum.columns[i]].div(train_final[world_type_dummy.columns[i]], 
                                                                                              fill_value=0)


In [81]:
train_final = train_final.replace([-np.inf, np.inf], np.nan)
train_final = train_final.fillna(0)

In [82]:
final_columns = ['installation_id', 'game_session', 'timestamp', 'world', 
                    'count_CRYSTALCAVES_Activity',
                    'count_CRYSTALCAVES_Assessment', 'count_CRYSTALCAVES_Clip',
                    'count_CRYSTALCAVES_Game', 'count_MAGMAPEAK_Activity',
                    'count_MAGMAPEAK_Assessment', 'count_MAGMAPEAK_Clip',
                    'count_MAGMAPEAK_Game', 'count_NONE_Clip', 'count_TREETOPCITY_Activity',
                    'count_TREETOPCITY_Assessment', 'count_TREETOPCITY_Clip',
                    'count_TREETOPCITY_Game',
                
                    'time_CRYSTALCAVES_Activity', 'time_CRYSTALCAVES_Assessment',
                    'time_CRYSTALCAVES_Clip', 'time_CRYSTALCAVES_Game',
                    'time_MAGMAPEAK_Activity', 'time_MAGMAPEAK_Assessment',
                    'time_MAGMAPEAK_Clip', 'time_MAGMAPEAK_Game', 'time_NONE_Clip',
                    'time_TREETOPCITY_Activity', 'time_TREETOPCITY_Assessment',
                    'time_TREETOPCITY_Clip', 'time_TREETOPCITY_Game', 
                 
                    'assessment_False',
                    'assessment_True', 'assessment_False_same_assess',
                    'assessment_True_same_assess',
                 
                    'event_code_2000', 'event_code_2010',
                    'event_code_2020', 'event_code_2025', 'event_code_2030',
                    'event_code_2035', 'event_code_2040', 'event_code_2050',
                    'event_code_2060', 'event_code_2070', 'event_code_2075',
                    'event_code_2080', 'event_code_2081', 'event_code_2083',
                    'event_code_3010', 'event_code_3020', 'event_code_3021',
                    'event_code_3110', 'event_code_3120', 'event_code_3121',
                    'event_code_4010', 'event_code_4020', 'event_code_4021',
                    'event_code_4022', 'event_code_4025', 'event_code_4030',
                    'event_code_4031', 'event_code_4035', 'event_code_4040',
                    'event_code_4045', 'event_code_4070',
                    'event_code_4080', 'event_code_4090', 'event_code_4095',
                    'event_code_4100', 'event_code_4110', 'event_code_4220',
                    'event_code_4230', 'event_code_4235', 'event_code_5000',
                    'event_code_5010',
#                     'event_code_4050',
                 
                    'assessment_False_five_hours', 'assessment_True_five_hours',
                    'assessment_five_hours_count',
                 
                    'title_12 Monkeys', 'title_Air Show',
                    'title_All Star Sorting', 'title_Balancing Act',
                    'title_Bird Measurer (Assessment)', 'title_Bottle Filler (Activity)',
                    'title_Bubble Bath', 'title_Bug Measurer (Activity)',
                    'title_Cart Balancer (Assessment)',
                    'title_Cauldron Filler (Assessment)', 'title_Chest Sorter (Assessment)',
                    'title_Chicken Balancer (Activity)', 'title_Chow Time',
                    'title_Costume Box', 'title_Crystal Caves - Level 1',
                    'title_Crystal Caves - Level 2', 'title_Crystal Caves - Level 3',
                    'title_Crystals Rule', 'title_Dino Dive', 'title_Dino Drink',
                    'title_Egg Dropper (Activity)', 'title_Fireworks (Activity)',
                    'title_Flower Waterer (Activity)', 'title_Happy Camel',
                    'title_Heavy, Heavier, Heaviest', 'title_Honey Cake',
                    'title_Leaf Leader', 'title_Lifting Heavy Things',
                    'title_Magma Peak - Level 1', 'title_Magma Peak - Level 2',
                    'title_Mushroom Sorter (Assessment)', 'title_Ordering Spheres',
                    'title_Pan Balance', "title_Pirate's Tale", 'title_Rulers',
                    'title_Sandcastle Builder (Activity)', 'title_Scrub-A-Dub',
                    'title_Slop Problem', 'title_Treasure Map',
                    'title_Tree Top City - Level 1', 'title_Tree Top City - Level 2',
                    'title_Tree Top City - Level 3', 'title_Watering Hole (Activity)',
                    'title_Welcome to Lost Lagoon!',
                 
                    'coverage_ratio',
                 
                    'time_period_afternoon', 'time_period_midnight', 
#                  'time_period_morning',
                    'time_period_night',
                 
                    'time_CRYSTALCAVES_Activity_avg',
                    'time_CRYSTALCAVES_Assessment_avg', 'time_CRYSTALCAVES_Clip_avg',
                    'time_CRYSTALCAVES_Game_avg', 'time_MAGMAPEAK_Activity_avg',
                    'time_MAGMAPEAK_Assessment_avg', 'time_MAGMAPEAK_Clip_avg',
                    'time_MAGMAPEAK_Game_avg', 'time_NONE_Clip_avg',
                    'time_TREETOPCITY_Activity_avg', 'time_TREETOPCITY_Assessment_avg',
                    'time_TREETOPCITY_Clip_avg', 'time_TREETOPCITY_Game_avg',
                 
                    'accuracy_group'
                ]

In [86]:
train_final = train_final[final_columns]
train_final[['world_CRYSTALCAVES', 'world_MAGMAPEAK', 'world_TREETOPCITY']] = pd.get_dummies(train_final.world).add_prefix('world_')
train_final = train_final.drop('world', axis=1)
train_final

,installation_id,game_session,timestamp,count_CRYSTALCAVES_Activity,count_CRYSTALCAVES_Assessment,count_CRYSTALCAVES_Clip,count_CRYSTALCAVES_Game,count_MAGMAPEAK_Activity,count_MAGMAPEAK_Assessment,count_MAGMAPEAK_Clip,...,time_MAGMAPEAK_Game_avg,time_NONE_Clip_avg,time_TREETOPCITY_Activity_avg,time_TREETOPCITY_Assessment_avg,time_TREETOPCITY_Clip_avg,time_TREETOPCITY_Game_avg,accuracy_group,world_CRYSTALCAVES,world_MAGMAPEAK,world_TREETOPCITY
0,07c25408,1c5ff4f9d45bddd1,2019-08-31 16:40:38.102000+00:00,2,0,2,1,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0,1,0,0
1,3bd55510,b8e726828d0aa231,2019-07-25 20:01:02.197000+00:00,0,0,0,0,0,0,0,...,0.000000,0.0,48360.500000,0.000000,0.0,61651.000000,3,0,0,1
2,3bd55510,657ef746a36dff81,2019-09-06 16:27:18.098000+00:00,0,0,1,0,4,0,4,...,112992.666667,0.0,62413.000000,27573.000000,0.0,86506.750000,3,0,1,0
3,484af773,94db1f8632919db7,2019-08-25 20:58:19.911000+00:00,0,0,0,0,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3,0,0,1
4,484af773,46af555036d99eb5,2019-08-25 21:15:10.178000+00:00,1,0,4,2,0,0,0,...,0.000000,0.0,0.000000,34607.000000,0.0,0.000000,3,1,0,0
5,484af773,fbc9e76d304fca9c,2019-08-25 21:24:30.891000+00:00,2,1,6,4,0,0,0,...,0.000000,0.0,0.000000,34607.000000,0.0,0.000000,1,1,0,0
6,51f0924c,a02a2465b0e1e24b,2019-07-29 00:04:29.915000+00:00,1,0,4,1,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3,1,0,0
7,51f0924c,221074eef693e944,2019-07-29 00:12:37.670000+00:00,2,1,6,3,0,0,0,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,3,1,0,0
8,9a898bba,ccac4050e0701270,2019-09-14 03:15:34.553000+00:00,0,0,4,0,0,0,2,...,0.000000,0.0,0.000000,0.000000,0.0,0.000000,1,0,0,1
9,9a898bba,7e3d76dadad1b380,2019-09-14 03:25:46.121000+00:00,0,0,6,2,0,0,2,...,0.000000,0.0,0.000000,70582.000000,0.0,0.000000,3,1,0,0


In [87]:
train_final.shape

(34, 142)

In [79]:
train_final.to_csv('train_data_sample.csv', index=False)

## Experiment

In [5]:
assignments = ['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']
train_data_assignment = train_data[train_data['title'].isin(assignments)]
train_data_assignment = train_data_assignment.reset_index(drop=True)


In [18]:
train_data_assignment.groupby(['installation_id', 'game_session']).last().shape

(21239, 9)

In [12]:
train_data_assignment[train_data_assignment.event_code == '4100']

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
43,25fa8af4,901acc108f55a5a1,2019-08-06 05:22:32.357000+00:00,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
164,25fa8af4,6bdf9623adc94d89,2019-08-06 05:38:08.036000+00:00,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,30,4100,18026,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
215,25fa8af4,9501794defd84e4d,2019-08-06 20:35:12.290000+00:00,"{""correct"":false,""stumps"":[3,2,5],""event_count...",0006a69f,29,4100,18484,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
224,25fa8af4,9501794defd84e4d,2019-08-06 20:35:16.846000+00:00,"{""correct"":true,""stumps"":[2,3,5],""event_count""...",0006a69f,38,4100,23043,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
258,070a5291,a9ef3ecb3d1acc6a,2019-08-06 20:50:33.283000+00:00,"{""correct"":true,""hats"":[4,8,5],""event_count"":3...",0006a69f,30,4100,34209,Bird Measurer (Assessment),Assessment,TREETOPCITY
280,392e14df,197a373a77101924,2019-09-14 15:35:54.361000+00:00,"{""buckets"":[0,0,0],""correct"":true,""buckets_pla...",0006c192,20,4100,12635,Cauldron Filler (Assessment),Assessment,MAGMAPEAK
330,25fa8af4,b2297d292892745a,2019-10-01 00:53:30.849000+00:00,"{""correct"":false,""stumps"":[0,0,0],""event_count...",0006c192,43,4100,32388,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
343,25fa8af4,b2297d292892745a,2019-10-01 00:53:36.596000+00:00,"{""correct"":false,""stumps"":[0,0,0],""event_count...",0006c192,56,4100,38139,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
363,25fa8af4,b2297d292892745a,2019-10-01 00:53:53.430000+00:00,"{""correct"":false,""stumps"":[2,5,3],""event_count...",0006c192,76,4100,54974,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
392,25fa8af4,b2297d292892745a,2019-10-01 00:54:18.450000+00:00,"{""correct"":false,""stumps"":[3,5,2],""event_count...",0006c192,105,4100,79992,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [17]:
df = df_copy.copy()

In [83]:
df[df.game_session == '901acc108f55a5a1']

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
2185,3bfd1a65,901acc108f55a5a1,2019-08-06 05:22:01.344000+00:00,0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2186,db02c830,901acc108f55a5a1,2019-08-06 05:22:01.400000+00:00,0006a69f,2,2025,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2187,a1e4395d,901acc108f55a5a1,2019-08-06 05:22:01.403000+00:00,0006a69f,3,3010,37,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2188,a52b92d5,901acc108f55a5a1,2019-08-06 05:22:05.242000+00:00,0006a69f,4,3110,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2189,a1e4395d,901acc108f55a5a1,2019-08-06 05:22:05.244000+00:00,0006a69f,5,3010,3901,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2190,28ed704e,901acc108f55a5a1,2019-08-06 05:22:07.812000+00:00,0006a69f,6,4025,6475,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2191,a52b92d5,901acc108f55a5a1,2019-08-06 05:22:07.812000+00:00,0006a69f,7,3110,6475,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2192,9d29771f,901acc108f55a5a1,2019-08-06 05:22:07.816000+00:00,0006a69f,8,3021,6475,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2193,c74f40cd,901acc108f55a5a1,2019-08-06 05:22:08.427000+00:00,0006a69f,9,3121,7084,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2194,28ed704e,901acc108f55a5a1,2019-08-06 05:22:09.742000+00:00,0006a69f,10,4025,8400,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [85]:
df[df.game_session == '6bdf9623adc94d89']

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
2768,3bfd1a65,6bdf9623adc94d89,2019-08-06 05:37:50.020000+00:00,0006a69f,1,2000,0,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2769,db02c830,6bdf9623adc94d89,2019-08-06 05:37:50.078000+00:00,0006a69f,2,2025,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2770,a1e4395d,6bdf9623adc94d89,2019-08-06 05:37:50.082000+00:00,0006a69f,3,3010,77,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2771,7da34a02,6bdf9623adc94d89,2019-08-06 05:37:52.799000+00:00,0006a69f,4,4070,2784,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2772,28ed704e,6bdf9623adc94d89,2019-08-06 05:37:53.631000+00:00,0006a69f,5,4025,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2773,a52b92d5,6bdf9623adc94d89,2019-08-06 05:37:53.632000+00:00,0006a69f,6,3110,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2774,9d29771f,6bdf9623adc94d89,2019-08-06 05:37:53.635000+00:00,0006a69f,7,3021,3625,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2775,c74f40cd,6bdf9623adc94d89,2019-08-06 05:37:54.253000+00:00,0006a69f,8,3121,4250,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2776,28ed704e,6bdf9623adc94d89,2019-08-06 05:37:54.930000+00:00,0006a69f,9,4025,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY
2777,9d29771f,6bdf9623adc94d89,2019-08-06 05:37:54.934000+00:00,0006a69f,10,3021,4925,Mushroom Sorter (Assessment),Assessment,TREETOPCITY


In [18]:
assignments = ['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']
df = df[df['title'].isin(assignments)]
df = df.reset_index(drop=True)

In [19]:
train_x_1 = df.groupby(['installation_id', 'game_session']).last().reset_index()
train_x_1 = train_x_1[['installation_id', 'game_session', 'event_count', 
                       'game_time', 'world']]

In [20]:
train_x_1

,installation_id,game_session,event_count,game_time,world
0,0006a69f,6bdf9623adc94d89,35,26827,TREETOPCITY
1,0006a69f,77b8ee947eb84b4e,87,92799,TREETOPCITY
2,0006a69f,901acc108f55a5a1,48,39803,TREETOPCITY
3,0006a69f,9501794defd84e4d,42,31843,TREETOPCITY
4,0006a69f,a9ef3ecb3d1acc6a,32,36368,TREETOPCITY
5,0006a69f,e7e7db2a241eadcc,17,8789,TREETOPCITY
6,0006c192,197a373a77101924,24,14469,MAGMAPEAK
7,0006c192,5f4c5e8c961afcf9,3,390,MAGMAPEAK
8,0006c192,957406a905d59afd,200,216374,TREETOPCITY
9,0006c192,b2297d292892745a,116,91828,TREETOPCITY


In [21]:
train_x_2 = pd.pivot_table(df, values='event_id', index=['installation_id', 'game_session'], columns=['event_code'], 
               aggfunc='count').add_prefix('event_code_').reset_index()

In [22]:
train_x_2 = train_x_2.fillna(0)

In [ ]:
train_y = pd.read_csv('train_labels.csv')

In [24]:
train_data = train_y.merge(train_x_1, on=['installation_id', 'game_session'], how='left')
train_data = train_data.merge(train_x_2, on=['installation_id', 'game_session'], how='left')
train_data

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group,event_count,game_time,world,...,event_code_4020,event_code_4025,event_code_4030,event_code_4035,event_code_4040,event_code_4070,event_code_4080,event_code_4090,event_code_4100,event_code_4110
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3,35,26827,TREETOPCITY,...,3.0,3.0,3.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0,87,92799,TREETOPCITY,...,0.0,22.0,22.0,0.0,0.0,4.0,0.0,0.0,0.0,11.0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3,48,39803,TREETOPCITY,...,6.0,3.0,8.0,0.0,2.0,6.0,0.0,0.0,1.0,0.0
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2,42,31843,TREETOPCITY,...,6.0,3.0,7.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3,32,36368,TREETOPCITY,...,3.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.000000,3,24,14469,MAGMAPEAK,...,3.0,0.0,3.0,0.0,0.0,7.0,0.0,0.0,1.0,0.0
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.500000,2,200,216374,TREETOPCITY,...,21.0,7.0,39.0,4.0,6.0,99.0,0.0,1.0,1.0,2.0
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.000000,0,116,91828,TREETOPCITY,...,8.0,3.0,14.0,0.0,6.0,53.0,0.0,0.0,4.0,0.0
8,ae691ec5ad5652cf,00129856,Bird Measurer (Assessment),1,0,1.000000,3,40,39701,TREETOPCITY,...,3.0,3.0,7.0,0.0,0.0,7.0,0.0,0.0,1.0,1.0
9,7b536271e99518f0,001d0ed0,Bird Measurer (Assessment),0,5,0.000000,0,51,52719,TREETOPCITY,...,0.0,10.0,11.0,0.0,0.0,9.0,0.0,0.0,0.0,5.0


In [25]:
event_code_submit = []
for index, row in train_data.iterrows():
    if row['title'] == 'Bird Measurer (Assessment)':
        event_code_submit.append(row['event_code_4110'])
    else:
        event_code_submit.append(row['event_code_4100'])
train_data['event_code_submit'] = event_code_submit

In [26]:
train_data = train_data[['title', 'event_count',
       'game_time', 'world', 'event_code_2000', 'event_code_2010',
       'event_code_2020', 'event_code_2025', 'event_code_2030',
       'event_code_2035', 'event_code_3010', 'event_code_3020',
       'event_code_3021', 'event_code_3110', 'event_code_3120',
       'event_code_3121', 'event_code_4020', 'event_code_4025',
       'event_code_4030', 'event_code_4035', 'event_code_4040',
       'event_code_4070', 'event_code_4080', 'event_code_4090',
       'event_code_4100', 'event_code_4110', 'event_code_submit', 'accuracy_group']]

In [160]:
train_y = train_data[['accuracy_group']]

In [27]:
train_data = pd.get_dummies(train_data)

In [33]:
train_data.columns = ['event_count', 'game_time', 'event_code_2000', 'event_code_2010',
       'event_code_2020', 'event_code_2025', 'event_code_2030',
       'event_code_2035', 'event_code_3010', 'event_code_3020',
       'event_code_3021', 'event_code_3110', 'event_code_3120',
       'event_code_3121', 'event_code_4020', 'event_code_4025',
       'event_code_4030', 'event_code_4035', 'event_code_4040',
       'event_code_4070', 'event_code_4080', 'event_code_4090',
       'event_code_4100', 'event_code_4110', 'event_code_submit',
       'accuracy_group', 'title_Bird_Measurer',
       'title_Cart_Balancer',
       'title_Cauldron_Filler', 'title_Chest_Sorter',
       'title_Mushroom_Sorter', 'world_CRYSTALCAVES',
       'world_MAGMAPEAK', 'world_TREETOPCITY']

In [34]:
train_data.to_csv('train_data.csv', index=False)

In [90]:
train_y

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.000000,3
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.500000,2
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.000000,0
8,ae691ec5ad5652cf,00129856,Bird Measurer (Assessment),1,0,1.000000,3
9,7b536271e99518f0,001d0ed0,Bird Measurer (Assessment),0,5,0.000000,0


## Submission

In [73]:
df = pd.read_csv('test.csv')
df.loc[:, 'timestamp'] = pd.to_datetime(df['timestamp'])
df.loc[:, 'event_code'] = df.event_code.astype('str')

In [74]:
df = df.drop('event_data', axis=1)
df_copy = df.copy()
df.head()

,event_id,game_session,timestamp,installation_id,event_count,event_code,game_time,title,type,world
0,27253bdc,0ea9ecc81a565215,2019-09-10 16:50:24.910000+00:00,00abaee7,1,2000,0,Welcome to Lost Lagoon!,Clip,NONE
1,27253bdc,c1ea43d8b8261d27,2019-09-10 16:50:55.503000+00:00,00abaee7,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
2,27253bdc,7ed86c6b72e725e2,2019-09-10 16:51:51.805000+00:00,00abaee7,1,2000,0,Magma Peak - Level 2,Clip,MAGMAPEAK
3,27253bdc,7e516ace50e7fe67,2019-09-10 16:53:12.825000+00:00,00abaee7,1,2000,0,Crystal Caves - Level 1,Clip,CRYSTALCAVES
4,7d093bf9,a022c3f60ba547e7,2019-09-10 16:54:12.115000+00:00,00abaee7,1,2000,0,Chow Time,Game,CRYSTALCAVES


In [17]:
df = df_copy.copy()

In [75]:
assignments = ['Bird Measurer (Assessment)', 'Cart Balancer (Assessment)', 'Cauldron Filler (Assessment)', 'Chest Sorter (Assessment)', 'Mushroom Sorter (Assessment)']
df = df[df['title'].isin(assignments)]
df = df.reset_index(drop=True)

In [76]:
test_x_1 = df.groupby(['installation_id', 'game_session']).last().reset_index()
test_x_1 = test_x_1[['installation_id', 'game_session', 'event_count', 
                       'game_time', 'title', 'world']]

In [77]:
test_x_2 = df.groupby(['installation_id', 'game_session']).first().reset_index()
test_x_1 = test_x_1.merge(test_x_2[['installation_id', 'game_session', 'timestamp']], on=['installation_id', 'game_session'], how='left')
test_x_1

,installation_id,game_session,event_count,game_time,title,world,timestamp
0,00abaee7,348d7f09f96af313,1,0,Cauldron Filler (Assessment),MAGMAPEAK,2019-09-12 13:52:12.193000+00:00
1,00abaee7,8b38fc0d2fd315dc,26,30038,Cart Balancer (Assessment),CRYSTALCAVES,2019-09-11 18:56:11.918000+00:00
2,01242218,009c890ce6c4f3e3,36,36926,Cauldron Filler (Assessment),MAGMAPEAK,2019-09-22 21:23:06.296000+00:00
3,01242218,1fef5d54cb4b775a,1,0,Cart Balancer (Assessment),CRYSTALCAVES,2019-10-09 20:23:16.209000+00:00
4,01242218,31423dbcd717919e,51,38746,Mushroom Sorter (Assessment),TREETOPCITY,2019-09-22 22:14:10.921000+00:00
5,01242218,597a8839a5a3468d,61,82149,Bird Measurer (Assessment),TREETOPCITY,2019-09-22 22:27:47.533000+00:00
6,01242218,ab61cae5e3215355,83,77638,Chest Sorter (Assessment),CRYSTALCAVES,2019-09-22 21:50:37.983000+00:00
7,01242218,e8e62de939f916bc,13,15839,Cart Balancer (Assessment),CRYSTALCAVES,2019-09-22 21:38:58.329000+00:00
8,017c5718,4b165a330a0bdd6c,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-09-21 11:28:21.757000+00:00
9,01a44906,be0b655ad1fee30c,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-07-27 16:28:10.394000+00:00


In [66]:
test_x_3 = pd.pivot_table(df, values='event_id', index=['installation_id', 'game_session'], columns=['event_code'], 
               aggfunc='count').add_prefix('event_code_').reset_index()

In [67]:
test_x_3 = test_x_3.fillna(0)

In [68]:
test_x_3

event_code,installation_id,game_session,event_code_2000,event_code_2010,event_code_2020,event_code_2025,event_code_2030,event_code_2035,event_code_3010,event_code_3020,...,event_code_4020,event_code_4025,event_code_4030,event_code_4035,event_code_4040,event_code_4070,event_code_4080,event_code_4090,event_code_4100,event_code_4110
0,00abaee7,348d7f09f96af313,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00abaee7,8b38fc0d2fd315dc,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,...,4.0,0.0,5.0,1.0,0.0,5.0,0.0,0.0,1.0,0.0
2,01242218,009c890ce6c4f3e3,1.0,1.0,2.0,0.0,2.0,0.0,4.0,1.0,...,4.0,1.0,4.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0
3,01242218,1fef5d54cb4b775a,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01242218,31423dbcd717919e,1.0,1.0,1.0,1.0,1.0,1.0,4.0,1.0,...,5.0,3.0,6.0,0.0,1.0,10.0,0.0,0.0,2.0,0.0
5,01242218,597a8839a5a3468d,1.0,1.0,2.0,0.0,2.0,0.0,4.0,3.0,...,5.0,8.0,14.0,0.0,0.0,7.0,0.0,0.0,2.0,3.0
6,01242218,ab61cae5e3215355,1.0,0.0,1.0,0.0,0.0,0.0,4.0,3.0,...,3.0,12.0,19.0,1.0,3.0,26.0,0.0,0.0,3.0,0.0
7,01242218,e8e62de939f916bc,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,017c5718,4b165a330a0bdd6c,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,01a44906,be0b655ad1fee30c,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
test_x_1 = test_x_1.merge(test_x_3, on=['installation_id', 'game_session'], how='left')
test_x_1

,installation_id,game_session,event_count,game_time,title,world,timestamp,event_code_2000,event_code_2010,event_code_2020,...,event_code_4020,event_code_4025,event_code_4030,event_code_4035,event_code_4040,event_code_4070,event_code_4080,event_code_4090,event_code_4100,event_code_4110
0,00abaee7,348d7f09f96af313,1,0,Cauldron Filler (Assessment),MAGMAPEAK,2019-09-12 13:52:12.193000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00abaee7,8b38fc0d2fd315dc,26,30038,Cart Balancer (Assessment),CRYSTALCAVES,2019-09-11 18:56:11.918000+00:00,1.0,1.0,1.0,...,4.0,0.0,5.0,1.0,0.0,5.0,0.0,0.0,1.0,0.0
2,01242218,009c890ce6c4f3e3,36,36926,Cauldron Filler (Assessment),MAGMAPEAK,2019-09-22 21:23:06.296000+00:00,1.0,1.0,2.0,...,4.0,1.0,4.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0
3,01242218,1fef5d54cb4b775a,1,0,Cart Balancer (Assessment),CRYSTALCAVES,2019-10-09 20:23:16.209000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01242218,31423dbcd717919e,51,38746,Mushroom Sorter (Assessment),TREETOPCITY,2019-09-22 22:14:10.921000+00:00,1.0,1.0,1.0,...,5.0,3.0,6.0,0.0,1.0,10.0,0.0,0.0,2.0,0.0
5,01242218,597a8839a5a3468d,61,82149,Bird Measurer (Assessment),TREETOPCITY,2019-09-22 22:27:47.533000+00:00,1.0,1.0,2.0,...,5.0,8.0,14.0,0.0,0.0,7.0,0.0,0.0,2.0,3.0
6,01242218,ab61cae5e3215355,83,77638,Chest Sorter (Assessment),CRYSTALCAVES,2019-09-22 21:50:37.983000+00:00,1.0,0.0,1.0,...,3.0,12.0,19.0,1.0,3.0,26.0,0.0,0.0,3.0,0.0
7,01242218,e8e62de939f916bc,13,15839,Cart Balancer (Assessment),CRYSTALCAVES,2019-09-22 21:38:58.329000+00:00,1.0,1.0,1.0,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,017c5718,4b165a330a0bdd6c,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-09-21 11:28:21.757000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,01a44906,be0b655ad1fee30c,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-07-27 16:28:10.394000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
test_x_1.columns

Index(['installation_id', 'game_session', 'event_count', 'game_time', 'title',
       'world', 'timestamp', 'event_code_2000', 'event_code_2010',
       'event_code_2020', 'event_code_2025', 'event_code_2030',
       'event_code_2035', 'event_code_3010', 'event_code_3020',
       'event_code_3021', 'event_code_3110', 'event_code_3120',
       'event_code_3121', 'event_code_4020', 'event_code_4025',
       'event_code_4030', 'event_code_4035', 'event_code_4040',
       'event_code_4070', 'event_code_4080', 'event_code_4090',
       'event_code_4100', 'event_code_4110'],
      dtype='object')

In [71]:
test_x_1.sort_values('timestamp', ascending=True).groupby('installation_id').last().reset_index()

,installation_id,game_session,event_count,game_time,title,world,timestamp,event_code_2000,event_code_2010,event_code_2020,...,event_code_4020,event_code_4025,event_code_4030,event_code_4035,event_code_4040,event_code_4070,event_code_4080,event_code_4090,event_code_4100,event_code_4110
0,00abaee7,348d7f09f96af313,1,0,Cauldron Filler (Assessment),MAGMAPEAK,2019-09-12 13:52:12.193000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,01242218,1fef5d54cb4b775a,1,0,Cart Balancer (Assessment),CRYSTALCAVES,2019-10-09 20:23:16.209000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,017c5718,4b165a330a0bdd6c,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-09-21 11:28:21.757000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,01a44906,be0b655ad1fee30c,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-07-27 16:28:10.394000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,01bc6cb6,46e8bbed71df7520,1,0,Cart Balancer (Assessment),CRYSTALCAVES,2019-09-06 18:05:26.197000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,02256298,73cf848935e13a2e,1,0,Cart Balancer (Assessment),CRYSTALCAVES,2019-08-25 19:16:14.447000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0267757a,363c252fbb51ba5a,1,0,Mushroom Sorter (Assessment),TREETOPCITY,2019-10-01 18:02:11.838000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,027e7ce5,3f92464665bbc7e8,1,0,Bird Measurer (Assessment),TREETOPCITY,2019-09-23 19:21:45.255000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,02a29f99,3d5b5a3897771980,1,0,Chest Sorter (Assessment),CRYSTALCAVES,2019-08-17 18:53:01.548000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0300c576,b1e50bb61bf9a4d4,1,0,Cart Balancer (Assessment),CRYSTALCAVES,2019-09-01 22:06:38.480000+00:00,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [86]:
test_y = pd.read_csv('sample_submission.csv')

In [95]:
id1 = test_y['installation_id']

In [94]:
id2 = train_y['installation_id']

In [24]:
train_data = train_y.merge(train_x_1, on=['installation_id', 'game_session'], how='left')
train_data = train_data.merge(train_x_2, on=['installation_id', 'game_session'], how='left')
train_data

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group,event_count,game_time,world,...,event_code_4020,event_code_4025,event_code_4030,event_code_4035,event_code_4040,event_code_4070,event_code_4080,event_code_4090,event_code_4100,event_code_4110
0,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3,35,26827,TREETOPCITY,...,3.0,3.0,3.0,0.0,0.0,5.0,0.0,0.0,1.0,0.0
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.000000,0,87,92799,TREETOPCITY,...,0.0,22.0,22.0,0.0,0.0,4.0,0.0,0.0,0.0,11.0
2,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.000000,3,48,39803,TREETOPCITY,...,6.0,3.0,8.0,0.0,2.0,6.0,0.0,0.0,1.0,0.0
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.500000,2,42,31843,TREETOPCITY,...,6.0,3.0,7.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.000000,3,32,36368,TREETOPCITY,...,3.0,3.0,6.0,0.0,0.0,2.0,0.0,0.0,1.0,1.0
5,197a373a77101924,0006c192,Cauldron Filler (Assessment),1,0,1.000000,3,24,14469,MAGMAPEAK,...,3.0,0.0,3.0,0.0,0.0,7.0,0.0,0.0,1.0,0.0
6,957406a905d59afd,0006c192,Bird Measurer (Assessment),1,1,0.500000,2,200,216374,TREETOPCITY,...,21.0,7.0,39.0,4.0,6.0,99.0,0.0,1.0,1.0,2.0
7,b2297d292892745a,0006c192,Mushroom Sorter (Assessment),0,4,0.000000,0,116,91828,TREETOPCITY,...,8.0,3.0,14.0,0.0,6.0,53.0,0.0,0.0,4.0,0.0
8,ae691ec5ad5652cf,00129856,Bird Measurer (Assessment),1,0,1.000000,3,40,39701,TREETOPCITY,...,3.0,3.0,7.0,0.0,0.0,7.0,0.0,0.0,1.0,1.0
9,7b536271e99518f0,001d0ed0,Bird Measurer (Assessment),0,5,0.000000,0,51,52719,TREETOPCITY,...,0.0,10.0,11.0,0.0,0.0,9.0,0.0,0.0,0.0,5.0


In [25]:
event_code_submit = []
for index, row in train_data.iterrows():
    if row['title'] == 'Bird Measurer (Assessment)':
        event_code_submit.append(row['event_code_4110'])
    else:
        event_code_submit.append(row['event_code_4100'])
train_data['event_code_submit'] = event_code_submit

In [26]:
train_data = train_data[['title', 'event_count',
       'game_time', 'world', 'event_code_2000', 'event_code_2010',
       'event_code_2020', 'event_code_2025', 'event_code_2030',
       'event_code_2035', 'event_code_3010', 'event_code_3020',
       'event_code_3021', 'event_code_3110', 'event_code_3120',
       'event_code_3121', 'event_code_4020', 'event_code_4025',
       'event_code_4030', 'event_code_4035', 'event_code_4040',
       'event_code_4070', 'event_code_4080', 'event_code_4090',
       'event_code_4100', 'event_code_4110', 'event_code_submit', 'accuracy_group']]

In [160]:
train_y = train_data[['accuracy_group']]

In [27]:
train_data = pd.get_dummies(train_data)

In [33]:
train_data.columns = ['event_count', 'game_time', 'event_code_2000', 'event_code_2010',
       'event_code_2020', 'event_code_2025', 'event_code_2030',
       'event_code_2035', 'event_code_3010', 'event_code_3020',
       'event_code_3021', 'event_code_3110', 'event_code_3120',
       'event_code_3121', 'event_code_4020', 'event_code_4025',
       'event_code_4030', 'event_code_4035', 'event_code_4040',
       'event_code_4070', 'event_code_4080', 'event_code_4090',
       'event_code_4100', 'event_code_4110', 'event_code_submit',
       'accuracy_group', 'title_Bird_Measurer',
       'title_Cart_Balancer',
       'title_Cauldron_Filler', 'title_Chest_Sorter',
       'title_Mushroom_Sorter', 'world_CRYSTALCAVES',
       'world_MAGMAPEAK', 'world_TREETOPCITY']

In [34]:
train_data.to_csv('train_data.csv', index=False)